In [14]:
import pandas as pd
import numpy as np
from scipy import stats

In [15]:
ticket_monster_df = pd.read_csv('tm_all_scenarios.csv')
pet_clinic_df = pd.read_csv('pc_all_scenarios_2.csv')

In [16]:
from datetime import datetime

# Define the datetime format
fmt = "%d-%m-%yT%H-%M-%S"

# Define the two timestamps
t1 = "11-12-23T03-19-36"
t2 = "09-12-23T06-28-39"

# Convert the timestamps to datetime objects
dt1 = datetime.strptime(t1, fmt)
dt2 = datetime.strptime(t2, fmt)

# Calculate the difference in hours
diff = dt1 - dt2
hours_diff = diff.total_seconds() / 3600

print("Difference in hours: ", hours_diff)

Difference in hours:  44.84916666666667


In [17]:
# Define a function to perform t-test for each scenario
def perform_ttest(df):
    # Get unique scenarios
    scenarios = df['ScenarioPath'].unique()

    # Initialize an empty DataFrame to store the results
    results = pd.DataFrame(columns=['Scenario', 't_statistic', 'p_value'])

    # Loop over each scenario
    for scenario in scenarios:
        # Filter data for current scenario
        scenario_data = df[df['ScenarioPath'] == scenario]

        # Split the data into two DataFrames, one for 'micro' and one for 'mono'
        micro_data = scenario_data[scenario_data['Architecture'] == 'micro']
        mono_data = scenario_data[scenario_data['Architecture'] == 'mono']

        # Perform t-test
        t_statistic, p_value = ttest_ind(micro_data['Joules'], mono_data['Joules'])

        # Append the results to the DataFrame
        results = results.append({'Scenario': scenario, 't_statistic': t_statistic, 'p_value': p_value}, ignore_index=True)

    return results

# Perform t-test for each scenario in ticket_monster_df and pet_clinic_df
ticket_monster_results = perform_ttest(ticket_monster_df)
pet_clinic_results = perform_ttest(pet_clinic_df)

# Print the results
print("Ticket Monster t-test Results:")
print(ticket_monster_results)
print("\nPet Clinic t-test Results:")
print(pet_clinic_results)

# first scenario, cant assume there's a difference
# second one 0.0000001 -> strong evidence that there is a difference
# third one, stronger evidence that there is a difference



Ticket Monster t-test Results:
   Scenario  t_statistic       p_value
0       1.0    -0.093993  9.254389e-01
1       2.0    -6.019273  1.266751e-07
2       3.0    -7.056458  2.356193e-09

Pet Clinic t-test Results:
   Scenario  t_statistic       p_value
0       1.0    -0.209252  8.349851e-01
1       2.0    -3.714823  4.590234e-04
2       3.0    -6.461945  2.333477e-08


/var/folders/w4/j0dx067d6_v_bwmcgq6v985r0000gn/T/ipykernel_16978/1170882671.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'Scenario': scenario, 't_statistic': t_statistic, 'p_value': p_value}, ignore_index=True)
/var/folders/w4/j0dx067d6_v_bwmcgq6v985r0000gn/T/ipykernel_16978/1170882671.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'Scenario': scenario, 't_statistic': t_statistic, 'p_value': p_value}, ignore_index=True)
/var/folders/w4/j0dx067d6_v_bwmcgq6v985r0000gn/T/ipykernel_16978/1170882671.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'Scenario': scenario, 't_statistic': t_statistic, 'p_value': p_value}, ignore_index=True

In [18]:
from scipy.stats import ttest_ind

# Split the data into two DataFrames, one for 'micro' and one for 'mono'
micro_data_pc = pet_clinic_df[pet_clinic_df['Architecture'] == 'micro']
mono_data_pc = pet_clinic_df[pet_clinic_df['Architecture'] == 'mono']

# Perform t-test
t_statistic, p_value = ttest_ind(micro_data_pc['Joules'], mono_data_pc['Joules'])

print("t statistic: ", t_statistic)
print("p-value: ", p_value)

t statistic:  -1.3180475268184224
p-value:  0.18918141708289368


In [19]:
from scipy.stats import f

# Calculate F test statistic
f_statistic = variance_micro_tm / variance_mono_tm

# Calculate degrees of freedom
df1 = len(ticket_monster_df[ticket_monster_df['Architecture'] == 'micro']) - 1
df2 = len(ticket_monster_df[ticket_monster_df['Architecture'] == 'mono']) - 1

# Calculate p-value
p_value = f.sf(f_statistic, df1, df2)

print("F statistic: ", f_statistic)
print("p-value: ", p_value)

F statistic:  0.7964127524008311
p-value:  0.8576011192381527


In [20]:
# Calculate F test statistic for PetClinic
f_statistic_pc = variance_micro_pc / variance_mono_pc

# Calculate degrees of freedom for PetClinic
df1_pc = len(pet_clinic_df[pet_clinic_df['Architecture'] == 'micro']) - 1
df2_pc = len(pet_clinic_df[pet_clinic_df['Architecture'] == 'mono']) - 1

# Calculate p-value for PetClinic
p_value_pc = f.sf(f_statistic_pc, df1_pc, df2_pc)

print("PetClinic F statistic: ", f_statistic_pc)
print("PetClinic p-value: ", p_value_pc)

PetClinic F statistic:  0.7166620562555076
PetClinic p-value:  0.9410290424295835
